## Dockerfile

- Contains how the application will run inside the container. Dockerfile is used in this project with details. Note that the Docker extension in VS Code helps more in writing the file.
- **Base Image**: Load the base image (e.g., `FROM node:14` for Node.js version 14 or `FROM python`). You can use `FROM` and the base image for your project.
  
  ```Dockerfile
  FROM baseImage
  ```

- **Working Directory**: Set the working directory for your application. This will create a directory for the app you want to dockerize.
  
  ```Dockerfile
  WORKDIR /app
  ```

- **Copy Files**: Use the following command to copy the `package.json` file to the working directory `/app`.
  
  ```Dockerfile
  COPY package.json .
  ```

- **Install Dependencies**: Run the command `npm install` to install all packages from `package.json`.

  ```Dockerfile
  RUN npm install
  ```

- **Copy All Files**: Copy the source files to the destination directory. The command `COPY . .` copies all files in the relative path to the working directory `/app`.
  
  ```Dockerfile
  COPY . .
  ```

- **Expose Port**: Document the port the application will use.
  
  ```Dockerfile
  EXPOSE 4000
  ```

- **Default Command**: Set the default command to be used when launching the container.
  
  ```Dockerfile
  CMD [ "npm", "start" ]
  ```

> **Note**: 
> - `RUN`: The command is triggered while building the Docker image.
> - `CMD`: The command is triggered when we launch the created Docker image.

## Docker Commands

- **Build Image**: Create an image from the Dockerfile. It is better to name the image using `-t "imagename"`. The `.` is used to build from the Dockerfile in the relative path.

  ```bash
  docker build -t express-node-app .
  ```

- **List Images**: To list all Docker images, use the following command:
  
  ```bash
  docker image ls
  ```

- **Run Container**: Use the following command to run the container from the image. It is good practice to give the container a name.
  
  ```bash
  docker run --name express-node-app-container -d express-node-app
  ```

  > `-d`: Run the container in detached mode (so the terminal is not taken over by the container).

- **Show Running Containers**: List all running containers with the following command:
  
  ```bash
  docker ps
  ```

- **Stop Container**: Stop the running container using this command:
  
  ```bash
  docker stop express-node-app-container
  ```

- **Remove Container**: Remove the container forcefully using the following command:
  
  ```bash
  docker rm express-node-app-container -f
  ```

- **Port Forwarding**: To forward the host port 4000 to the container port 4000, use this command:
  
  ```bash
  docker run --name express-node-app-container -d -p 4000:4000 express-node-app
  ```

- **Access Container Terminal**: To access the container's terminal, run:
  
  ```bash
  docker exec -it express-node-app-container bash
  ```

  > `-it`: Interactive terminal.  
  > `express-node-app-container`: Name of the container.  
  > The command uses the bash shell.

- **Check Logs**: View the logs of the running container using this command:
  
  ```bash
  docker logs express-node-app-container
  ```

## General Notes

- Docker Hub (https://hub.docker.com) contains all the images you need, and it is open-source, meaning you can also publish your own images, similar to GitHub.

- A Dockerfile describes how the application will run inside the container.

- You can create multiple containers from a single image.

- Docker images consist of layers (e.g., Layer 1 is `node:14`, Layer 2 could be app files, etc.).

- Each container can hold a service, and the same app can run on multiple containers, with each service on its own container.

- **Node.js application note**: Nodemon is used in `devDependencies` to make the dev server listen for changes and reload automatically during development.

## Docker Optimization

- It is possible to create non-root users in your Dockerfile to avoid running everything as root.

- A `.dockerignore` file specifies which files will be ignored during the image build and container runtime.

- **Command Sequence Optimization**:
  
  ```Dockerfile
  COPY package.json .
  RUN npm install
  COPY . .
  ```

  By separating `package.json` from copying all files, Docker can utilize caching mechanisms. This ensures that `npm install` is triggered only when there are changes in `package.json`, thus preventing unnecessary reinstallation of packages when other files change.

- Environment variables are stored in a `.env` file.

- **Optimization Tip**: Limiting which files are copied is an optimization step to reduce build time and storage space.

## Docker Hot Reload

- Any changes in the local directory will not be reflected in the container unless the image is rebuilt and the container is rerun, which is impractical.

- We used volumes with binding (local_dire:container_dir) to apply hot reload

- **Hot reload** allows syncing between local and container files (mirroring the local directory with the container directory).

- Hot reload can be achieved using the `-v` flag (from volumes) and the absolute path of the local directory and container directory.

  ```bash
  docker run --name express-node-app-container -v F:/DevOps/docker/my-express-app:/app -d -p 4000:4000 express-node-app
  ```

> **Disadvantages of hot reload**: Any files created or removed inside the container will automatically be reflected in the local directory.
